In [7]:
# Import von Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [8]:
# Input - Parameter
Pv_kWp = 10
Zinsatz = 0.06
Systemkosten = 800
Betriebskosten = 10 
Lebensdauer = 20


dfEinspeisung = pd.read_csv("PV_Einspeisung.csv")
dfEinspeisung.rename(columns = {0:'Leistung'}, inplace = True)

dfPeise2023 = pd.read_csv("ENTSOE_prices_for_2023_in_ct_per_kWh.csv")
#dfPeise2023_AT = dfPeise2023.split(";")[0]